<div class="alert alert-info">
<u><strong>Authors:</strong></u> <b>Ahmed Mukhtar</b> (ahmed.mukhtar@mail.polimi.it),and <b>Ahmed Yassin</b> (ahmedmohamed1@mail.polimi.it) - 2023 - Politecnico di Milano, Italy <br>
</div>

## Stations' Climate Zone Class per Month

In [ ]:
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
%load_ext autoreload

Select the year.

In [ ]:
year_w = widgets.Dropdown(
    options = [i for i in range(2014, 2024)],
    value = 2022,
    description = 'Year:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
year_w

In [ ]:
year = year_w.value

Select the month.

In [ ]:
month_w = widgets.Dropdown(
    options = [i for i in range(1, 13)],
    value = 2,
    description = 'Month:',
    disabled = False,
    layout = {'width': 'max-content'},
    style = {'description_width': 'initial'}
)
month_w

In [ ]:
month = month_w.value

Set the **output folders** where the data will be saved. 

In [ ]:
netatmo_out_path = 'Netatmo_csv_files/'

In [ ]:
Netatmo_data = pd.read_csv(netatmo_out_path + f'temp_Net_milan_{year}-{month}_filtered.csv', skiprows=0)

In [ ]:
stations_classes = pd.read_csv(netatmo_out_path + f'Stations_LCZ_csv/Stations_LCZ_2023-{month}.csv', skiprows=0)

In [ ]:
# Define the mapping for LCZ_Class
lcz_class_mapping = {
    2: '2 - Compact midrise',
    3: '3 - Compact low-rise',
    5: '5 - Open midrise',
    6: '6 - Open low-rise',
    8: '8 - Large low-rise',
    101: '101 - Dense trees',
    102: '102 - Scattered trees',
    104: '104 - Low Plants',
    105: '105 - Bare rock or paved',
    106: '106 - Bare soil or sand',
    107: '107 - Water',
    
}

stations_classes.rename(columns={'_majority': 'LCZ'}, inplace=True)

# Create the 'LCZ_Class' column using the mapping
stations_classes['LCZ_Class'] = stations_classes['LCZ'].map(lcz_class_mapping)

stations_classes.drop(columns=['_minority'], inplace=True)

In [ ]:
# Merge df1 and df2 on 'module_id' to add 'LCZ' column and keep only stations with 'LCZ' class
classified_df = pd.merge(Netatmo_data, stations_classes[['module_id', 'LCZ', 'LCZ_Class']], on='module_id', how='inner')

#reoder the columns
classified_df = classified_df[['time','Temperature','min_temp','max_temp',
                               'device_id','module_id','LCZ','LCZ_Class','lat','long',
                               'timezone','country','altitude','city','street','geometry']]

# Convert the 'time' column to datetime to facilitate filtering
classified_df['time'] = pd.to_datetime(classified_df['time'])

# Extract the hours from the 'time' column
classified_df['hour'] = classified_df['time'].dt.hour

# Define the daytime and nighttime hours for each month
if month == 2:  # February
    daytime_hours = [14, 15, 16]
    nighttime_hours = [21, 22, 23]
elif month == 3:  # March
    daytime_hours = [14, 15, 16]
    nighttime_hours = [22, 23, 00]
elif month == 4:  # April
    daytime_hours = [14, 15, 16]
    nighttime_hours = [23, 00, 1]
elif month in [5, 6, 8]: # May, June, August 
    daytime_hours = [14, 15, 16]
    nighttime_hours = [00, 1, 2]
elif month == 7: # July 
    daytime_hours = [16, 17, 18]
    nighttime_hours = [00, 1, 2]

# Filter the data for daytime and nighttime hours
daytime_df = classified_df[classified_df['hour'].isin(daytime_hours)].copy()
daytime_df['period'] = 'daytime'
nighttime_df = classified_df[classified_df['hour'].isin(nighttime_hours)].copy()
nighttime_df['period'] = 'nighttime'

# Concatenate the daytime and nighttime dataframes
day_night_time_df = pd.concat([daytime_df, nighttime_df])


# Drop the 'hour' column as it is no longer needed
day_night_time_df = day_night_time_df.drop(columns=['hour'])

#reoder the columns
day_night_time_df = day_night_time_df[['time','period','Temperature','min_temp','max_temp',
                               'device_id','module_id','LCZ','LCZ_Class','lat','long',
                               'timezone','country','altitude','city','street','geometry']]




# Save the classified dataframe to a CSV file
classified_df.to_csv(netatmo_out_path + f'Stations_Classified/temp_Net_milan_{year}-{month}_classified.csv', index=False)
day_night_time_df.to_csv(netatmo_out_path + f'Stations_Classified/Daytime-Nighttime_Temp_per_month/day_night_temp_{year}-{month}.csv', index=False)

In [ ]:
#classified_df

In [ ]:
#day_night_time_df

In [ ]:
# List of LCZ_Class values to be removed
lcz_classes_to_remove = [
    '101 - Dense trees',
    '102 - Scattered trees',
    '104 - Low Plants',
    '105 - Bare rock or paved',
    '106 - Bare soil or sand',
    '107 - Water'
]

# Filter the dataframe to exclude rows with the specified LCZ_Class values
filtered_daytime = daytime_df[~daytime_df['LCZ_Class'].isin(lcz_classes_to_remove)]
filtered_nighttime = nighttime_df[~nighttime_df['LCZ_Class'].isin(lcz_classes_to_remove)]

In [ ]:
# Group by 'LCZ_Class' and get all temperatures for daytime and nighttime
daytime_all = filtered_daytime[['LCZ_Class', 'Temperature']].copy()
daytime_all['period'] = 'daytime'

nighttime_all = filtered_nighttime[['LCZ_Class', 'Temperature']].copy()
nighttime_all['period'] = 'nighttime'

# Combine both dataframes into a single dataframe for plotting
combined = pd.concat([daytime_all, nighttime_all])
combined = combined.reset_index(drop=True)

In [ ]:
# Sort the dataframe by 'LCZ_Class'
sorted_data = combined.sort_values('LCZ_Class')

# Create the box plot with separate boxes for daytime and nighttime(2)
plt.figure(figsize=(16, 12))
sns.boxplot(x='LCZ_Class', y='Temperature', hue='period', data=sorted_data)
plt.title(f'Daytime and Nighttime Temperatures by Local Climate Zone (LCZ) {month}/{year}', fontsize=22)
plt.xticks(rotation=90, fontsize=16)
plt.xlabel('Local Climate Zone (LCZ)', fontsize=18)
plt.ylabel('Temperature (°C)', fontsize=18)
plt.yticks(fontsize=16)
#plt.yticks(range(10, 30, 1))  # Setting Y-axis ticks with an interval of 2
plt.legend(title='Period', title_fontsize=18, fontsize=16)

# Save the plot as an image (e.g., PNG, JPG, PDF)
plt.savefig(netatmo_out_path + f'Stations_Classified/Box_plots/Daytime&Nighttime-T-by-LCZ-{year}_{month}.png', bbox_inches='tight')
plt.show()
